In [2]:
from src.grdb.database.v1_1_0 import sample, preparation_step, dal, Base, mdf_forge, properties, recipe, raman_set
from src.gresq.config import config
import pandas as pd

In [3]:
dal.init_db(config['development'])

AttributeError: 'NoneType' object has no attribute '_instantiate_plugins'

In [3]:
Base.metadata.drop_all(bind=dal.engine)
Base.metadata.create_all(bind=dal.engine)

In [4]:
var_map = pd.read_csv('data/varmap.csv').to_dict()
data = pd.read_csv('data/recipestest.csv').iloc[:-1,:]

In [5]:
col_names = data.columns
with dal.session_scope() as session:
    for i in range(data.shape[0]):
        s = sample()
        s.reference = data.iloc[i,-1]
        for j in range(31):
            value = data.iloc[i,j]
            if pd.isnull(value) == False:
                dbkey = var_map[col_names[j]][0]
                setattr(s,dbkey,value)
        session.add(s)
        session.commit()
        # Annealing
        for step,j in enumerate(range(32,110,13)):
            prep = preparation_step()
            prep.name = "Annealing"
            prep.sample_id = s.id
            prep.step = step
            for p in range(13):
                value = data.iloc[i,j+p]
                dbkey = var_map[col_names[j+p]][0]
                if pd.isnull(value) == False:
#                     print(dbkey,value)
                    setattr(prep,dbkey,value)
            session.add(prep)
        # Growing
        for step,j in enumerate(range(110,189,13)):
            prep = preparation_step()
            prep.name = "Growing"
            prep.sample_id = s.id
            prep.step = step
            for p in range(13):
                value = data.iloc[i,j+p]
                dbkey = var_map[col_names[j+p]][0]
                if pd.isnull(value) == False:
#                     print(dbkey,value)
                    setattr(prep,dbkey,value)
            session.add(prep)
        # Cooling
        for step,j in enumerate(range(191,269,13)):
            prep = preparation_step()
            prep.name = "Cooling"
            prep.cooling_rate = data.iloc[i,190]
            prep.sample_id = s.id
            prep.step = step
            for p in range(13):
                value = data.iloc[i,j+p]
                dbkey = var_map[col_names[j+p]][0]
                if pd.isnull(value) == False:
#                     print(dbkey,value)
                    setattr(prep,dbkey,value)
            session.add(prep)
        session.commit()

In [24]:
with dal.session_scope() as session:
    for q in session.query(sample).\
        join(preparation_step,sample.preparation_steps)\
        .filter(preparation_step.furnace_temperature>800):
        print(q)

In [7]:
session.close()

In [8]:
[c for c in data.columns if 'Cooling' in c]

['PREPARATION STEP DETAIL: Cooling Rate ($^\\circ$C/s)']

In [9]:
import sqlalchemy
sqlalchemy.__version__

'1.1.11'